# Qubit Pure Dephasing: Bloch-Redfield vs Lindblad
We study a qubit with Hamiltonian $H = rac{mega_0}{2} igma_z$ coupled to a bosonic bath inducing pure dephasing.
We compare QuTiP's `brmesolve` (full Redfield) and `mesolve` with a Lindblad dephasing operator. We monitor density matrix eigenvalues to detect CP violations (if any).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import basis, sigmaz, sigmax, Qobj, brmesolve, mesolve, BosonicEnvironment, tensor
from qutip import OhmicEnvironment
from diagnostics import density_matrix_positivity, trajectory_min_eigs
from plotting import style, plot_min_eig
style()
w0 = 1.0  # qubit splitting
H = 0.5 * w0 * sigmaz()
# pure dephasing coupling operator A = sigma_z (commutes with H)
A = sigmaz()
# Ohmic bath providing S(0) > 0 giving dephasing rate.
bath = OhmicEnvironment(T=0.5, wc=10.0, alpha=0.05, s=1.0)
psi0 = basis(2,0)
tlist = np.linspace(0,10,300)
# Bloch-Redfield evolution
res_br = brmesolve(H, psi0, tlist, a_ops=[[A, bath.power_spectrum]], e_ops=[])
# Lindblad model with rate gamma_phi = S(0)
gamma_phi = bath.power_spectrum(0)
res_me = mesolve(H, psi0, tlist, c_ops=[np.sqrt(gamma_phi/2)*A], e_ops=[])
min_eigs_br = trajectory_min_eigs(res_br.states)
min_eigs_me = trajectory_min_eigs(res_me.states)
plt.figure(figsize=(6,3))
plt.plot(tlist, min_eigs_br, label='BR')
plt.plot(tlist, min_eigs_me, label='ME', ls='--')
plt.axhline(0,color='k',lw=1)
plt.ylabel(r'$\lambda_{min}$')
plt.xlabel('t')
plt.legend(frameon=False)
plt.title('Minimal eigenvalue (positivity check)')
plt.show()